In [1]:
import configparser
import os
import pyspark
from pyspark import SparkConf
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.types as T
from pyspark.sql.functions import udf, col, lit
from pyspark.sql.types import MapType, StringType, FloatType
from pyspark.sql import DataFrame, Window
from collections import OrderedDict
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

In [2]:
config = configparser.ConfigParser()
config.read("capstone.cfg")

os.chdir(config["PATH"]["project"])
project_path = config["PATH"]["project"]


In [3]:
def create_spark_session():
    spark = SparkSession \
        .builder \
        .appName("covid_DB") \
        .getOrCreate()
    
    return spark

In [4]:
spark = create_spark_session()

In [5]:
df_covid = spark.read.option("header", True).csv(os.path.join(project_path, "DATA","COVID", "us-counties.csv"))

In [6]:
df_covid.printSchema()

root
 |-- date: string (nullable = true)
 |-- county: string (nullable = true)
 |-- state: string (nullable = true)
 |-- fips: string (nullable = true)
 |-- cases: string (nullable = true)
 |-- deaths: string (nullable = true)



In [7]:
df_covid = df_covid.withColumn("date", F.to_date("date"))

In [8]:
df_covid.printSchema()

root
 |-- date: date (nullable = true)
 |-- county: string (nullable = true)
 |-- state: string (nullable = true)
 |-- fips: string (nullable = true)
 |-- cases: string (nullable = true)
 |-- deaths: string (nullable = true)



In [9]:
df_covid.show()

+----------+-----------+----------+-----+-----+------+
|      date|     county|     state| fips|cases|deaths|
+----------+-----------+----------+-----+-----+------+
|2020-01-21|  Snohomish|Washington|53061|    1|     0|
|2020-01-22|  Snohomish|Washington|53061|    1|     0|
|2020-01-23|  Snohomish|Washington|53061|    1|     0|
|2020-01-24|       Cook|  Illinois|17031|    1|     0|
|2020-01-24|  Snohomish|Washington|53061|    1|     0|
|2020-01-25|     Orange|California|06059|    1|     0|
|2020-01-25|       Cook|  Illinois|17031|    1|     0|
|2020-01-25|  Snohomish|Washington|53061|    1|     0|
|2020-01-26|   Maricopa|   Arizona|04013|    1|     0|
|2020-01-26|Los Angeles|California|06037|    1|     0|
|2020-01-26|     Orange|California|06059|    1|     0|
|2020-01-26|       Cook|  Illinois|17031|    1|     0|
|2020-01-26|  Snohomish|Washington|53061|    1|     0|
|2020-01-27|   Maricopa|   Arizona|04013|    1|     0|
|2020-01-27|Los Angeles|California|06037|    1|     0|
|2020-01-2

In [10]:
df_covid.count()

1170376

In [11]:
df_covid.where(col("fips").isNull()).count()

10721

In [12]:
map_fips_stations = spark.read.parquet( os.path.join(project_path, "OUT_DATA/", "stations_per_fips"))

In [13]:
map_fips_stations.printSchema()

root
 |-- location_id: long (nullable = true)
 |-- fips: string (nullable = true)
 |-- county: string (nullable = true)
 |-- fips_state: string (nullable = true)
 |-- station_id: string (nullable = true)
 |-- station_name: string (nullable = true)
 |-- station_state: string (nullable = true)
 |-- angle: double (nullable = true)
 |-- distance: double (nullable = true)
 |-- measured: string (nullable = true)



In [14]:
nyt_locations = df_covid.select("fips", "county", "state").distinct()

In [15]:
nyt_locations.count()

3274

In [16]:
map_locations = map_fips_stations.select("location_id", "fips", "county", "fips_state").distinct()
map_locations.count()

3272

In [17]:
nyt_locations.join(map_locations, 
            (nyt_locations.fips.eqNullSafe(map_locations.fips) ) &
            (nyt_locations.county.eqNullSafe(map_locations.county) )& 
            (nyt_locations.state.eqNullSafe( map_locations.fips_state) ) )\
            .count()

3272

In [18]:
nyt_locations.join(map_locations, 
            (nyt_locations.fips.eqNullSafe(map_locations.fips) ) &
            (nyt_locations.county.eqNullSafe(map_locations.county) )& 
            (nyt_locations.state.eqNullSafe( map_locations.fips_state) ) ,
            how = "left_anti")\
            .show()

+-----+--------------------+------+
| fips|              county| state|
+-----+--------------------+------+
|02997|Bristol Bay plus ...|Alaska|
|02998|Yakutat plus Hoon...|Alaska|
+-----+--------------------+------+



In [19]:
# filter out non matched locations and add location_id
df_covid_filter = df_covid.alias("covid").join( map_locations.alias("loc"),
            (df_covid.fips.eqNullSafe(map_fips_stations.fips) ) &
            (df_covid.county.eqNullSafe(map_fips_stations.county) ) & 
            (df_covid.state.eqNullSafe( map_fips_stations.fips_state) ) )\
       .select("date", "location_id", "covid.fips", "covid.county", "covid.state", "deaths", "cases")
                                             

In [20]:
# compute daily cases and daily deaths, from cumulated values
# first add a column with the lag value (i.e. the value from the previous day)
# then compute the difference btw current day and previous day values
w = Window.partitionBy("location_id").orderBy("date")
df_covid_daily = df_covid_filter\
    .withColumn("deaths_prev", F.lag("deaths", count = 1, default = 0).over(w) ) \
    .withColumn("cases_prev", F.lag("cases", count = 1, default = 0).over(w) )\
    .withColumn("daily_deaths", col("deaths") - col("deaths_prev"))\
    .withColumn("daily_cases", col("cases") - col("cases_prev"))

In [21]:
df_covid_daily.printSchema()

root
 |-- date: date (nullable = true)
 |-- location_id: long (nullable = true)
 |-- fips: string (nullable = true)
 |-- county: string (nullable = true)
 |-- state: string (nullable = true)
 |-- deaths: string (nullable = true)
 |-- cases: string (nullable = true)
 |-- deaths_prev: string (nullable = true)
 |-- cases_prev: string (nullable = true)
 |-- daily_deaths: double (nullable = true)
 |-- daily_cases: double (nullable = true)



In [41]:
out_path = os.path.join(project_path, "OUT_DATA", "covid_per_county")

df_covid_daily.select("date", "location_id", "daily_cases", "daily_deaths") \
    .write.mode("overwrite").parquet(out_path)

In [42]:
max_date = df_covid_daily.agg({"date" : "max" }).collect()[0]["max(date)"]

In [44]:
max_date

datetime.date(2021, 3, 29)

In [35]:
# compute last date for each location_id : used to filter out data for subsequent loads
w = Window.partitionBy("location_id").orderBy( col("date").desc())
columns = df_covid_filter.columns[:]
last_data = df_covid_filter.withColumn("rank", F.rank().over(w))\
    .where("rank == 1")\
    .select(columns)

In [40]:
last_data.write\
    .format("jdbc")\
    .option("url", "jdbc:postgresql:capstone")\
    .option("dbtable", "last_data")\
    .option("user","postgres")\
    .option("password", "postgres")\
    .save()

In [39]:
last_data.count()

3272